# Project Planning Stage (Individual)

In [1]:
# Run this cell before continuing
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

# *(1) Data Description*: Players Data

- Number of Observations: 197
- Number of Variables: 7
- Name and Type of Variables: experience (chr), hashedEmail (chr), name (chr), gender (chr),
  played_hours (dbl), Age (dbl), subscribe (lgl)
- Variable Meanings:
  - experience: How adept the player was at the game seperated into 5 ranks: beginner, amateur, regular, pro, and veteran
  - hashedEmail: Each player's email identity
  - name: The name of the player
  - gender: The gender of the player (male, female, or other)
  - played_hours: The number of hours spent playing the game for each player
  - Age: The ages of the players
  - subscribe: If the player was subscribed to the channel for the game (true or false)
- Any Issues in Data: there are some empty spots found in the data
- Issues that can't be directly seen: idk...
- How data was collected: Data was collected from a research group in Computer Science conducted by Frank Wood by using PlaiCraft as the minecraft server to record player's actions and navigate through the world

- Summary statistics for the players data:

In [6]:
players <- read_csv("data/players.csv")
summary(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


  experience        subscribe       hashedEmail         played_hours    
 Length:196         Mode :logical   Length:196         Min.   :  0.000  
 Class :character   FALSE:52        Class :character   1st Qu.:  0.000  
 Mode  :character   TRUE :144       Mode  :character   Median :  0.100  
                                                       Mean   :  5.846  
                                                       3rd Qu.:  0.600  
                                                       Max.   :223.100  
                                                                        
     name              gender               Age       
 Length:196         Length:196         Min.   : 9.00  
 Class :character   Class :character   1st Qu.:17.00  
 Mode  :character   Mode  :character   Median :19.00  
                                       Mean   :21.14  
                                       3rd Qu.:22.75  
                                       Max.   :58.00  
                               

# *(1) Data Description*: Sessions Data

- Number of Observations: 1535
- Number of Variables: 5
- Name and Type of Variables: hashedEmail (chr), start_time (chr), end_time (chr),
  original_start_time (dbl), original_end_time (dbl)
- Variable Meanings:
  - hashedEmail: Each player's email identity
  - start_time: The exact start date of the player when they first started in the game (formatted as day/month/year)
  - end_time: The exact end date of the player when they stopped playing the game (formatted as day/month/year)
  - original_start_time: The start time of the player 
  - original_end_time: The end time of the player
- Any Issues in Data: The original start and end time numbers are extremely large
- Issues that can't be directly seen: idk...
- How data was collected: Data was collected from a research group in Computer Science conducted by Frank Wood by using PlaiCraft as the minecraft server to record player's actions and navigate through the world

Summary statistics for sessions data:

In [5]:
sessions <- read_csv("data/sessions.csv")
summary(sessions)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 hashedEmail         start_time          end_time         original_start_time
 Length:1535        Length:1535        Length:1535        Min.   :1.712e+12  
 Class :character   Class :character   Class :character   1st Qu.:1.716e+12  
 Mode  :character   Mode  :character   Mode  :character   Median :1.719e+12  
                                                          Mean   :1.719e+12  
                                                          3rd Qu.:1.722e+12  
                                                          Max.   :1.727e+12  
                                                                             
 original_end_time  
 Min.   :1.712e+12  
 1st Qu.:1.716e+12  
 Median :1.719e+12  
 Mean   :1.719e+12  
 3rd Qu.:1.722e+12  
 Max.   :1.727e+12  
 NA's   :2          